In [1]:
%matplotlib inline
from time import time
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import sklearn.linear_model
import sklearn.model_selection
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error 

from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor  

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

### First of all, lets explore our dataset, how it looks and any patterns we can suggest from it manualy

In [2]:
data = pd.read_csv("books_train.csv")

data.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,26237,Revolutionary Girl Utena Vol. 3: To Sprout,Chiho Saito/Be-Pas,4.05,1591162076,9781591162070,eng,200,1153,16,2/4/2004,VIZ Media LLC
1,33448,Positioning: The Battle for Your Mind,Al Ries/Jack Trout,4.04,0071359168,9780071359160,en-US,246,126,9,1/18/2001,McGraw-Hill Education
2,13739,Twelve Fair Kingdoms,Suzette Haden Elgin,3.99,0425058506,9780425058503,eng,195,141,10,3/1/1983,Berkley
3,2711,The Riverside Chaucer,Geoffrey Chaucer/Larry Dean Benson/F.N. Robinson,4.18,0395290317,9780395290316,enm,1327,7760,152,12/12/1987,Houghton Mifflin
4,40540,PHP and MySQL Web Development (Developer's Lib...,Luke Welling/Laura Thomson,3.96,0672326728,752063326725,en-US,1008,590,31,10/1/2004,Sams


In [3]:
test = pd.read_csv("books_test.csv")

test.head()

,bookID,title,authors,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,2538,El hombre duplicado,José Saramago/Pilar del Río,8466312803,9788466312806,spa,380,1295,106,9/1/2004,Punto de Lectura
1,31912,Buffy the Vampire Slayer and Philosophy: Fear ...,James B. South/William Irwin,0812695313,9780812695311,eng,335,2519,85,3/13/2003,Open Court
2,7716,Plays Pleasant,George Bernard Shaw/Dan H. Laurence/W.J. McCor...,0140437940,9780140437942,eng,336,265,10,3/27/2003,Penguin Classics
3,7811,A Friend of the Earth,T. Coraghessan Boyle,0747553467,9780747553465,eng,275,30,3,10/8/2001,Bloomsbury Paperbacks
4,19379,Mark Twain's Own Autobiography: The Chapters f...,Mark Twain/Michael J. Kiskis,0299125408,9780299125400,eng,301,9,1,10/1/1990,University of Wisconsin Press


### So, now we can understood that average rating is 3.93. Let's try submit this rating as first attempt and understood the score for it

In [4]:
train_average_rating_mean = data["average_rating"].mean()

first_attempt_result = test["bookID"].to_frame()
first_attempt_result["average_rating"] = train_average_rating_mean
first_attempt_result.head()

,bookID,average_rating
0,2538,3.932359
1,31912,3.932359
2,7716,3.932359
3,7811,3.932359
4,19379,3.932359


In [5]:
first_attempt_result.to_csv("first_attempt.csv", encoding='utf-8', index=False)

### And the result is quite bad

![first_attempt_score](attempt_1/first_attempt.png "Title")


This score is the mean score error

In [6]:
le = preprocessing.LabelEncoder()
data = data.drop(data.index[data['num_pages'] >= 1000])
data = data.drop(data.index[data['ratings_count'] >= 1000000])
data = data.drop(data.index[data['text_reviews_count'] >= 20000])

data['title'] = le.fit_transform(data['title'])
data['authors'] = le.fit_transform(data['authors'])
data['publication_date'] = le.fit_transform(data['publication_date'])
data['publisher'] = le.fit_transform(data['publisher'])
# enc_lang = pd.get_dummies(data['language_code'])
# data = pd.concat([data, enc_lang], axis = 1)

data.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,26237,4279,729,4.05,1591162076,9781591162070,eng,200,1153,16,1256,1770
1,33448,4097,45,4.04,0071359168,9780071359160,en-US,246,126,9,138,1058
2,13739,7199,4647,3.99,0425058506,9780425058503,eng,195,141,10,1298,204
5,40806,7194,115,3.83,0099429160,9780099429166,eng,275,7545,388,1773,1792
6,9936,703,2854,3.46,0385280793,9780385280792,eng,304,727,23,2363,448


In [7]:
X = data.drop(['average_rating', 'language_code', 'isbn', 'isbn13'], axis = 1)
y = data['average_rating']

In [8]:
X.describe()

,bookID,title,authors,num_pages,ratings_count,text_reviews_count,publication_date,publisher
count,8152.000000,8152.000000,8152.000000,8152.000000,8152.000000,8152.000000,8152.000000,8152.000000
mean,21445.166340,3855.356845,2612.614450,317.270486,12259.957924,420.074215,1542.562561,991.569676
std,13093.374797,2223.636043,1494.230419,183.204245,48425.820951,1346.481974,907.572717,575.990938
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10526.500000,1936.000000,1318.750000,192.000000,102.000000,9.000000,738.750000,464.000000
50%,20716.000000,3855.500000,2610.000000,292.000000,749.000000,47.000000,1571.000000,1035.500000
75%,32296.250000,5790.250000,3910.000000,406.000000,4863.750000,232.000000,2324.250000,1501.250000
max,45641.000000,7716.000000,5198.000000,999.000000,978535.000000,19846.000000,3089.000000,1928.000000


In [9]:
le = preprocessing.LabelEncoder()

test['title'] = le.fit_transform(test['title'])
test['authors'] = le.fit_transform(test['authors'])
test['publication_date'] = le.fit_transform(test['publication_date'])
test['publisher'] = le.fit_transform(test['publisher'])
# enc_lang = pd.get_dummies(test['language_code'])
# test = pd.concat([test, enc_lang], axis = 1)

test = test.drop(['language_code', 'isbn', 'isbn13'], axis = 1)

test.head()

,bookID,title,authors,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,2538,556,1078,380,1295,106,1424,695
1,31912,321,875,335,2519,85,685,597
2,7716,1368,649,336,265,10,731,639
3,7811,47,1917,275,30,3,294,130
4,19379,1100,1303,301,9,1,149,915


### Let's try the simplest linear regression

In [10]:
def build_and_save_result(prediction, file_name):
    result = pd.DataFrame(columns = ["bookID", "average_rating"])

    result["bookID"] = test["bookID"]
    result["average_rating"] = pd.Series(prediction)

    result.to_csv(file_name, encoding='utf-8', index=False)

In [13]:
lr = LinearRegression()
lr.fit(X, y)

prediction = lr.predict(test)

build_and_save_result(prediction, "linear_regression.csv")

### Let's try the Decisition Tree Regressor approach for this problem

Based on my study/course 2-3 years ago
https://github.com/Almaz-KG/MachineLearning/blob/develop/coursera/intro-ml/week-1/practice/w1-practice-03.ipynb

##### First of all, I think it's obvious to select several valuable columns from input data, instead of selecting all columns

In [62]:
regressor = DecisionTreeRegressor()
#                                   random_state = 211,
#                                   min_samples_split=10,
#                                   max_features="auto", 
#                                   max_depth=15)
regressor.fit(X, y) 

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [64]:
desition_tree_prediction = pd.Series(regressor.predict(test))
build_and_save_result(desition_tree_prediction, "desition_tree_prediction.csv")

pd.Series(desition_tree_prediction).describe()

count    2781.000000
mean        3.940910
std         0.330984
min         0.000000
25%         3.790000
50%         3.950000
75%         4.150000
max         5.000000
dtype: float64

In [33]:
pd.Series(y).describe()

count    8152.000000
mean        3.927004
std         0.348140
min         0.000000
25%         3.770000
50%         3.950000
75%         4.130000
max         5.000000
Name: average_rating, dtype: float64

### I've submited this attempt and the result is worser than a simple average

![second_attempt_score](attempt_2/second_attempt_decisition_tree_regression.png "Title")

### Let's try the KNeighborsRegressor approach for this problem

### This attempt based on my previous practical work (2 years ago) (Boston Housing Price Regression)

https://github.com/Almaz-KG/MachineLearning/blob/develop/coursera/intro-ml/week-2/practice/w2-practice-02--Boston-housing-regression.ipynb

In [53]:
scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = scaler.fit_transform(X)
x_train = pd.DataFrame(x_train_scaled)

knn_model = KNeighborsRegressor(n_neighbors = 7)
knn_model.fit(x_train, y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                    weights='uniform')

In [54]:
test_scaled = scale(test)

knn_prediction = knn_model.predict(test_scaled)
build_and_save_result(knn_prediction, "knn_regressor.csv")

### I've submited this attempt and the result is slightly improved

![third_attempt_score](attempt_3/third_attempt_k_neighbors_regressor.png "Title")

### Let's try Random Forest approach

In [46]:
scaler = MinMaxScaler(feature_range=(0, 1))

x_train_scaled = scaler.fit_transform(X)
x_train = pd.DataFrame(x_train_scaled)

rf_model = RandomForestRegressor(n_estimators=3, random_state=3)
rf_model.fit(x_train_scaled, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=3, n_jobs=None, oob_score=False,
                      random_state=3, verbose=0, warm_start=False)

In [47]:
test_scaled = scale(test)

rf_prediction = rf_model.predict(test_scaled)
build_and_save_result(rf_prediction, "random_forest_regressor.csv")